In [8]:
import pandas as pd
import numpy as np

## Parsing Weights & impacts

In [9]:
def parse_weights(df,weights,impacts):
    if len(impacts.split(',')) != df.shape[1]-1 or len(weights.split(',')) != df.shape[1]-1:
        raise ValueError("Number of impacts and weights must match the number of columns in the DataFrame.")

    w = [float(weight) for weight in weights.split(",")]
    imp = [1 if impact == '+' else 0 for impact in impacts.split(",")]
    
    return w,imp


## Making Normalized Decesion Matrix

In [10]:
def normalize_dec_mat(df,w):
    rsumsq=0
    for i,col in enumerate(df.columns[1:]):
        rsumsq=((df[col]**2).sum())**0.5
        df.loc[:df.shape[0] - 1, col] /= rsumsq/w[i]
    return df

## Ideal Vpos & Vneg of each column

In [11]:
def calculate_ideal_solutions(norm_df, impacts):
    vpos = [] 
    vneg = [] 

    for i, col in enumerate(norm_df.columns[1:]):
        if impacts[i] == 1: 
            vpos.append(norm_df[col].max())
            vneg.append(norm_df[col].min())
        else:
            vpos.append(norm_df[col].min())
            vneg.append(norm_df[col].max())

    return vpos, vneg

## Calculate Topsis Score

In [12]:
def calculate_topsis_scores(norm_df, vpos, vneg):
    spos = []
    sneg = []

    for i in range(len(norm_df)):
        spos.append(sum((norm_df.iloc[i, 1:] - vpos) ** 2) ** 0.5)
        sneg.append(sum((norm_df.iloc[i, 1:] - vneg) ** 2) ** 0.5)

    scores = [sneg[i] / (sneg[i] + spos[i]) for i in range(len(spos))]
    return scores

## TOPSIS

In [13]:
def topsis(df,weights,impacts):
    w,imp=parse_weights(df,weights,impacts)
    df=normalize_dec_mat(df,w)
    vpos,vneg=calculate_ideal_solutions(df,imp)
    scores = calculate_topsis_scores(df, vpos, vneg)
    df['Topsis Score'] = scores
    df['Rank'] = df['Topsis Score'].rank(ascending=False).astype(int)
    return df
    

In [16]:
df = pd.read_excel('data.xlsx')
weights = "0.25,0.25,0.25,0.25,0.25"
impacts = "-,+,+,+,+"
topsis(df, weights, impacts)

,Fund Name,P1,P2,P3,P4,P5,Topsis Score,Rank
0,M1,0.087769,0.086100,0.105358,0.080635,0.083998,0.547124,3
1,M2,0.095083,0.100652,0.110076,0.060715,0.067452,0.484122,5
2,M3,0.082545,0.075186,0.075481,0.089445,0.088268,0.444211,7
3,M4,0.081500,0.073973,0.100641,0.081209,0.083731,0.500429,4
4,M5,0.098218,0.106716,0.056610,0.119133,0.112820,0.599571,2
5,M6,0.091949,0.093376,0.102213,0.098639,0.099477,0.685336,1
6,M7,0.068962,0.053358,0.083343,0.093659,0.092271,0.463212,6
7,M8,0.097173,0.104290,0.053465,0.070867,0.070388,0.365797,8
